In [20]:
import sys
import os
import json
import pandas as pd
import numpy as np
import ast
import itertools
import random
import copy
from datetime import datetime
from collections import Counter, defaultdict

import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

sys.path.append("../")

from src.helpers import io
from src.classes.dataset import Dataset
from src.classes.annotation_set import AnnotationSet

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
# FILL IN:
PATH_TO_DATASET = "../data/static/wildchat4k-raw.json"
DATASET_ID = "wildchat_1m"
PATH_TO_ANNOTATIONS_DIR = "../res/gpto3mini-json-wildchat"
OUTDIR = "data/annotation_analysis_v0/time_geography"
os.makedirs(OUTDIR, exist_ok=True)

# Load dataset (w/o annotations)
dataset = Dataset.load(PATH_TO_DATASET)

# Load annotations into dataset
for fpath in io.listdir_nohidden(PATH_TO_ANNOTATIONS_DIR):
    annotation_set = AnnotationSet.load_automatic(path=fpath, source="automatic_v0")
    dataset.add_annotations(annotation_set)

prompt-multi_turn_relationship: 0 / 10127 failed due to invalid annotations.
prompt-interaction_features: 0 / 10127 failed due to invalid annotations.
turn-sensitive_use_flags: 0 / 10127 failed due to invalid annotations.
turn-topic: 1 / 10127 failed due to invalid annotations.
response-interaction_features: 0 / 10127 failed due to invalid annotations.
prompt-function_purpose: 6 / 10127 failed due to invalid annotations.
prompt-media_format: 0 / 10127 failed due to invalid annotations.
response-media_format: 0 / 10127 failed due to invalid annotations.
response-answer_form: 0 / 10127 failed due to invalid annotations.


# Research Questions: How do dialogue characteristics differ over time and geography across the whole set of conversations?

Characteristics:
- Prompt: 
    - "function_purpose", 
    - ("level_id": "prompt")"interaction_features", 
    - ("level_id": "prompt")"media_format", 
    - (Multi-turn relationships?)
- Response: 
    - "answer_form", 
    - ("level_id": "response")"interaction_features", 
    - ("level_id": "response")"media_format"
- Turn: 
    - "topic", 
    - "sensitive_use_flags"
- Conversation: 
    - "model", 
    - (Languages?, Conversation length/stats?)


# Temporal Analysis

In [33]:
# Helper: extract month from timestamp
def extract_month(timestamp):
    try:
        dt = datetime.fromisoformat(timestamp.replace("Z", "+00:00"))
        return dt.strftime("%Y-%m")
    except Exception:
        return None

# Temporal analysis function
def analyze_temporal_distribution(label_key, level):
    monthly_counts = defaultdict(Counter)

    for conv in dataset.data:
        month = extract_month(conv.time)
        if not month:
            continue

        for msg in conv.conversation:
            if level == "conversation":
                if msg.turn == 0:  # log only once per conversation
                    label = getattr(conv, label_key, None)
                    if label:
                        labels = label if isinstance(label, list) else [label]
                        for l in labels:
                            monthly_counts[month][l] += 1
                break

            elif level == "turn" and f"automatic_v0-turn_{label_key}" in msg.metadata:
                label = msg.metadata[f"automatic_v0-turn_{label_key}"].value
                labels = label if isinstance(label, list) else [label]
                for l in labels:
                    monthly_counts[month][l] += 1

            elif level == "prompt" and msg.role == "user" and f"automatic_v0-prompt_{label_key}" in msg.metadata:
                label = msg.metadata[f"automatic_v0-prompt_{label_key}"].value
                labels = label if isinstance(label, list) else [label]
                for l in labels:
                    monthly_counts[month][l] += 1

            elif level == "response" and msg.role == "assistant" and f"automatic_v0-response_{label_key}" in msg.metadata:
                label = msg.metadata[f"automatic_v0-response_{label_key}"].value
                labels = label if isinstance(label, list) else [label]
                for l in labels:
                    monthly_counts[month][l] += 1

    # Convert to DataFrame
    df_counts = pd.DataFrame(monthly_counts).fillna(0).T
    df_counts.index.name = "Month"
    df_counts.sort_index(inplace=True)

    # Ensure numeric and drop empty
    df_counts = df_counts.apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)
    df_counts = df_counts.loc[(df_counts != 0).any(axis=1)]

    if df_counts.empty:
        print(f"[Skipped] No data available to plot for {label_key} @ {level}")
        return

    # Normalize to percent
    df_pct = df_counts.div(df_counts.sum(axis=1), axis=0) * 100

    # Combine into formatted table
    df_formatted = df_counts.astype(str) + " (" + df_pct.round(1).astype(str) + "%)"

    print(f"\n=== Monthly Distribution Table for {label_key} @ {level} ===")
    display(df_formatted)

    # Plot
    fig, ax = plt.subplots(figsize=(14, 7))
    df_pct.plot(kind="bar", stacked=True, ax=ax, colormap="tab20")
    plt.title(f"Monthly Distribution of {label_key.replace('_', ' ').title()}")
    plt.ylabel("Percentage")
    plt.xlabel("Month")
    plt.xticks(rotation=45, ha="right")
    ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), title="Labels", fontsize='small', title_fontsize='medium')
    plt.tight_layout()
    plot_path = os.path.join(OUTDIR, f"temporal_{label_key}_{level}.png")
    plt.savefig(plot_path, bbox_inches="tight")
    plt.close()
    print(f"Saved plot: {plot_path}")

# List of all dimensions to analyze
targets = [
    ("function_purpose", "prompt"),
    ("interaction_features", "prompt"),
    ("media_format", "prompt"),
    ("answer_form", "response"),
    ("interaction_features", "response"),
    ("media_format", "response"),
    ("topic", "turn"),
    ("sensitive_use_flags", "turn"),
    ("model", "conversation"),
]

# Run all analyses
for label_key, level in targets:
    analyze_temporal_distribution(label_key, level)


=== Monthly Distribution Table for function_purpose @ prompt ===


,Information analysis (Content explanation / interpretation),"Advice, Guidance, & Recommendations (Activity / product recommendations)","Advice, Guidance, & Recommendations (Instructions / How-to)",Information retrieval (general info from web),Content generation (other),Translation (language to language),"Content generation (general prose, discussion or explanation)",Content generation (creative / fiction writing),Content generation (code),No clear task,...,Editorial & formatting (Natural language style or re-formatting),Role-play / social simulation (platonic companion / friend),Information analysis (Content quality review or assessment),Editorial & formatting (Code content editing),"Advice, Guidance, & Recommendations (Action planning (scheduling, robotics))",Role-play / social simulation (simulation of real person / celebrity),Information analysis (Other content analysis / description),Information analysis (Content Classification),Information analysis (Ranking or Scoring),Role-play / social simulation (therapist / coach)
Month,,,,,,,,,,,,,,,,,,,,,
2023-04,61 (6.1%),17 (1.7%),87 (8.7%),101 (10.1%),23 (2.3%),16 (1.6%),67 (6.7%),162 (16.2%),111 (11.1%),121 (12.1%),...,8 (0.8%),13 (1.3%),3 (0.3%),5 (0.5%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%)
2023-05,113 (11.8%),9 (0.9%),90 (9.4%),119 (12.5%),21 (2.2%),15 (1.6%),70 (7.3%),114 (11.9%),59 (6.2%),110 (11.5%),...,6 (0.6%),6 (0.6%),2 (0.2%),4 (0.4%),1 (0.1%),3 (0.3%),2 (0.2%),0 (0.0%),0 (0.0%),0 (0.0%)
2023-06,52 (6.0%),4 (0.5%),63 (7.3%),94 (10.8%),27 (3.1%),24 (2.8%),54 (6.2%),165 (19.0%),59 (6.8%),133 (15.3%),...,14 (1.6%),5 (0.6%),0 (0.0%),2 (0.2%),0 (0.0%),0 (0.0%),0 (0.0%),1 (0.1%),1 (0.1%),1 (0.1%)
2023-07,54 (7.9%),2 (0.3%),46 (6.7%),73 (10.7%),25 (3.7%),17 (2.5%),43 (6.3%),174 (25.5%),45 (6.6%),59 (8.6%),...,5 (0.7%),5 (0.7%),2 (0.3%),0 (0.0%),0 (0.0%),3 (0.4%),1 (0.1%),0 (0.0%),0 (0.0%),0 (0.0%)
2023-08,56 (8.8%),4 (0.6%),25 (3.9%),82 (12.9%),16 (2.5%),40 (6.3%),50 (7.9%),108 (17.0%),24 (3.8%),44 (6.9%),...,5 (0.8%),5 (0.8%),0 (0.0%),1 (0.2%),0 (0.0%),1 (0.2%),0 (0.0%),0 (0.0%),1 (0.2%),1 (0.2%)
2023-09,32 (5.4%),4 (0.7%),35 (5.9%),64 (10.8%),15 (2.5%),23 (3.9%),43 (7.2%),103 (17.3%),41 (6.9%),37 (6.2%),...,9 (1.5%),1 (0.2%),0 (0.0%),3 (0.5%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),1 (0.2%),0 (0.0%)
2023-10,53 (8.9%),7 (1.2%),36 (6.0%),62 (10.4%),24 (4.0%),20 (3.4%),70 (11.7%),71 (11.9%),26 (4.4%),48 (8.1%),...,1 (0.2%),1 (0.2%),1 (0.2%),1 (0.2%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%)
2023-11,52 (6.8%),1 (0.1%),52 (6.8%),65 (8.5%),15 (2.0%),12 (1.6%),56 (7.3%),173 (22.6%),57 (7.4%),51 (6.6%),...,5 (0.7%),4 (0.5%),1 (0.1%),5 (0.7%),0 (0.0%),2 (0.3%),0 (0.0%),2 (0.3%),0 (0.0%),0 (0.0%)
2023-12,78 (9.8%),1 (0.1%),58 (7.3%),102 (12.8%),13 (1.6%),21 (2.6%),49 (6.1%),132 (16.5%),41 (5.1%),94 (11.8%),...,19 (2.4%),8 (1.0%),1 (0.1%),1 (0.1%),0 (0.0%),2 (0.3%),0 (0.0%),10 (1.3%),0 (0.0%),0 (0.0%)


Saved plot: data/annotation_analysis_v0/time_geography/temporal_function_purpose_prompt.png

=== Monthly Distribution Table for interaction_features @ prompt ===


,None,Courtesy/Politeness,Role-assignment,Jailbreak attempt,Reinforcement/Praise/Scolding,Companionship,Reinforcement/Praise
Month,,,,,,,
2023-04,891 (88.4%),61 (6.1%),32 (3.2%),8 (0.8%),13 (1.3%),3 (0.3%),0 (0.0%)
2023-05,808 (82.8%),61 (6.2%),61 (6.2%),25 (2.6%),20 (2.0%),1 (0.1%),0 (0.0%)
2023-06,753 (85.1%),58 (6.6%),40 (4.5%),17 (1.9%),12 (1.4%),4 (0.5%),1 (0.1%)
2023-07,563 (79.9%),50 (7.1%),64 (9.1%),10 (1.4%),16 (2.3%),2 (0.3%),0 (0.0%)
2023-08,473 (73.8%),77 (12.0%),79 (12.3%),7 (1.1%),4 (0.6%),1 (0.2%),0 (0.0%)
2023-09,486 (79.5%),60 (9.8%),56 (9.2%),4 (0.7%),3 (0.5%),2 (0.3%),0 (0.0%)
2023-10,458 (74.2%),49 (7.9%),95 (15.4%),8 (1.3%),6 (1.0%),1 (0.2%),0 (0.0%)
2023-11,638 (80.9%),70 (8.9%),55 (7.0%),12 (1.5%),7 (0.9%),7 (0.9%),0 (0.0%)
2023-12,679 (82.5%),54 (6.6%),64 (7.8%),11 (1.3%),13 (1.6%),2 (0.2%),0 (0.0%)


Saved plot: data/annotation_analysis_v0/time_geography/temporal_interaction_features_prompt.png

=== Monthly Distribution Table for media_format @ prompt ===


...

Saved plot: data/annotation_analysis_v0/time_geography/temporal_topic_turn.png

=== Monthly Distribution Table for sensitive_use_flags @ turn ===


,None,Sexually explicit content (Other),Criminal planning or other suspected illegal activity not listed elsewhere,"Inciting violence, hateful or other harmful behavior (harassment & bullying)","Discriminatory practices (Misrepresentation, stereotyping, or inappropriate reference to sensitive attributes)","Inciting violence, hateful or other harmful behavior (physical harm)",Sexually explicit content (fictitious person),Sexually explicit content (Request/discussion of CSAM),"Inciting violence, hateful or other harmful behavior (self-harm)",Privacy concerns (Possible identifiable information),...,Sexually explicit content (real person),Generating defamatory content,Weapons & drugs,"Cyberattacks (enabling/enacting malware, computer viruses, worms, malicious code, ...)",Impersonation attempts,Cyberattacks,"Privacy concerns (Possible sensitive information) (e.g., API keys, passwords, other confidential information)",Potential violation of external policy / ethics,Criminal planning or other suspected illegal activity not listed,Other
Month,,,,,,,,,,,,,,,,,,,,,
2023-04,924 (92.6%),40 (4.0%),1 (0.1%),8 (0.8%),3 (0.3%),7 (0.7%),13 (1.3%),1 (0.1%),1 (0.1%),0 (0.0%),...,0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%)
2023-05,801 (82.8%),98 (10.1%),3 (0.3%),25 (2.6%),16 (1.7%),4 (0.4%),10 (1.0%),0 (0.0%),0 (0.0%),2 (0.2%),...,5 (0.5%),1 (0.1%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%)
2023-06,729 (83.5%),71 (8.1%),8 (0.9%),15 (1.7%),3 (0.3%),14 (1.6%),17 (1.9%),2 (0.2%),3 (0.3%),0 (0.0%),...,7 (0.8%),0 (0.0%),3 (0.3%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%)
2023-07,585 (85.5%),46 (6.7%),0 (0.0%),11 (1.6%),1 (0.1%),9 (1.3%),21 (3.1%),3 (0.4%),0 (0.0%),0 (0.0%),...,0 (0.0%),0 (0.0%),1 (0.1%),5 (0.7%),2 (0.3%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%)
2023-08,543 (86.5%),32 (5.1%),0 (0.0%),8 (1.3%),4 (0.6%),12 (1.9%),17 (2.7%),6 (1.0%),0 (0.0%),1 (0.2%),...,0 (0.0%),0 (0.0%),0 (0.0%),1 (0.2%),0 (0.0%),1 (0.2%),1 (0.2%),0 (0.0%),0 (0.0%),0 (0.0%)
2023-09,552 (92.9%),20 (3.4%),0 (0.0%),5 (0.8%),1 (0.2%),4 (0.7%),6 (1.0%),3 (0.5%),1 (0.2%),1 (0.2%),...,0 (0.0%),0 (0.0%),0 (0.0%),1 (0.2%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%)
2023-10,547 (92.4%),23 (3.9%),1 (0.2%),6 (1.0%),1 (0.2%),1 (0.2%),6 (1.0%),0 (0.0%),0 (0.0%),0 (0.0%),...,4 (0.7%),0 (0.0%),0 (0.0%),2 (0.3%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%)
2023-11,671 (87.3%),34 (4.4%),3 (0.4%),16 (2.1%),2 (0.3%),8 (1.0%),22 (2.9%),3 (0.4%),0 (0.0%),6 (0.8%),...,1 (0.1%),2 (0.3%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%)
2023-12,692 (86.6%),55 (6.9%),11 (1.4%),17 (2.1%),5 (0.6%),2 (0.3%),11 (1.4%),1 (0.1%),0 (0.0%),0 (0.0%),...,2 (0.3%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),1 (0.1%),1 (0.1%),0 (0.0%)


Saved plot: data/annotation_analysis_v0/time_geography/temporal_sensitive_use_flags_turn.png

=== Monthly Distribution Table for model @ conversation ===


,gpt-3.5-turbo-0301,gpt-4-0314,gpt-3.5-turbo-0613,gpt-4-1106-preview,gpt-3.5-turbo-0125,gpt-4-0125-preview
Month,,,,,,
2023-04,218 (70.8%),90 (29.2%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%)
2023-05,246 (79.9%),62 (20.1%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%)
2023-06,239 (77.6%),38 (12.3%),31 (10.1%),0 (0.0%),0 (0.0%),0 (0.0%)
2023-07,0 (0.0%),0 (0.0%),308 (100.0%),0 (0.0%),0 (0.0%),0 (0.0%)
2023-08,0 (0.0%),0 (0.0%),308 (100.0%),0 (0.0%),0 (0.0%),0 (0.0%)
2023-09,0 (0.0%),0 (0.0%),308 (100.0%),0 (0.0%),0 (0.0%),0 (0.0%)
2023-10,0 (0.0%),0 (0.0%),308 (100.0%),0 (0.0%),0 (0.0%),0 (0.0%)
2023-11,0 (0.0%),0 (0.0%),267 (86.7%),41 (13.3%),0 (0.0%),0 (0.0%)
2023-12,0 (0.0%),0 (0.0%),180 (58.4%),128 (41.6%),0 (0.0%),0 (0.0%)


Saved plot: data/annotation_analysis_v0/time_geography/temporal_model_conversation.png


# Geographic Analysis

In [34]:
def extract_country(geo_string):
    if not geo_string:
        return None
    return geo_string.split(";")[0].strip()

def analyze_geographic_distribution(label_key, level, top_n=15):
    geo_counts = defaultdict(Counter)

    for conv in dataset.data:
        country = extract_country(conv.geography)
        if not country:
            continue

        for msg in conv.conversation:
            if level == "conversation":
                if msg.turn == 0:
                    label = getattr(conv, label_key, None)
                    if label:
                        labels = label if isinstance(label, list) else [label]
                        for l in labels:
                            geo_counts[country][l] += 1
                break

            elif level == "turn" and f"automatic_v0-turn_{label_key}" in msg.metadata:
                label = msg.metadata[f"automatic_v0-turn_{label_key}"].value
                labels = label if isinstance(label, list) else [label]
                for l in labels:
                    geo_counts[country][l] += 1

            elif level == "prompt" and msg.role == "user" and f"automatic_v0-prompt_{label_key}" in msg.metadata:
                label = msg.metadata[f"automatic_v0-prompt_{label_key}"].value
                labels = label if isinstance(label, list) else [label]
                for l in labels:
                    geo_counts[country][l] += 1

            elif level == "response" and msg.role == "assistant" and f"automatic_v0-response_{label_key}" in msg.metadata:
                label = msg.metadata[f"automatic_v0-response_{label_key}"].value
                labels = label if isinstance(label, list) else [label]
                for l in labels:
                    geo_counts[country][l] += 1

    # Convert to DataFrame
    df_counts = pd.DataFrame(geo_counts).fillna(0).T
    df_counts.index.name = "Country"
    df_counts.sort_index(inplace=True)

    # Keep only top countries
    df_counts['__total__'] = df_counts.sum(axis=1)
    df_top = df_counts.sort_values('__total__', ascending=False).head(top_n).drop(columns="__total__")

    if df_top.empty:
        print(f"[Skipped] No data available to plot for {label_key} @ {level}")
        return

    # Normalize to percent
    df_pct = df_top.div(df_top.sum(axis=1), axis=0) * 100

    # Combine into formatted table
    df_formatted = df_top.astype(int).astype(str) + " (" + df_pct.round(1).astype(str) + "%)"

    # Print table
    print(f"\n=== Geographic Distribution Table for {label_key} @ {level} ===")
    display(df_formatted)

    # Plot
    fig, ax = plt.subplots(figsize=(14, 8))
    df_pct.plot(kind="bar", stacked=True, ax=ax, colormap="tab20")

    plt.title(f"{'Geographic' if 'Country' in df_pct.index.name else 'Monthly'} Distribution of {label_key.replace('_', ' ').title()}")
    plt.ylabel("Percentage")
    plt.xlabel(df_pct.index.name)
    plt.xticks(rotation=45, ha="right")
    ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), title="Labels", fontsize='small', title_fontsize='medium')
    plt.tight_layout()
    plot_path = os.path.join(OUTDIR, f"{'geo' if 'Country' in df_pct.index.name else 'temporal'}_{label_key}_{level}.png")
    plt.savefig(plot_path, bbox_inches='tight')  # Ensure legend is not cut off
    plt.close()
    print(f"Saved plot: {plot_path}")


# Full set of dimensions to analyze
targets = [
    ("function_purpose", "prompt"),
    ("interaction_features", "prompt"),
    ("media_format", "prompt"),
    ("answer_form", "response"),
    ("interaction_features", "response"),
    ("media_format", "response"),
    ("topic", "turn"),
    ("sensitive_use_flags", "turn"),
    ("model", "conversation"),
]

for label_key, level in targets:
    analyze_geographic_distribution(label_key, level)


=== Geographic Distribution Table for function_purpose @ prompt ===


,Information analysis (Content explanation / interpretation),"Advice, Guidance, & Recommendations (Activity / product recommendations)","Advice, Guidance, & Recommendations (Instructions / How-to)",Information retrieval (general info from web),Content generation (code),Content generation (academic / essay writing),"Content generation (general prose, discussion or explanation)",Editorial & formatting (Content summarization),No clear task,Content generation (administrative writing),...,Content generation (prompts for another AI system),Role-play / social simulation (platonic companion / friend),Content generation (code documentation),"Reasoning (Verbal problems, logic games, puzzles or riddles)","Advice, Guidance, & Recommendations (Social and personal advice)",Editorial & formatting (Code style and re-formatting),Information analysis (Content Classification),Information analysis (Ranking or Scoring),Information analysis (Other content analysis / description),"Advice, Guidance, & Recommendations (Action planning (scheduling, robotics))"
Country,,,,,,,,,,,,,,,,,,,,,
United States,100 (5.5%),10 (0.6%),73 (4.0%),164 (9.1%),57 (3.1%),46 (2.5%),125 (6.9%),19 (1.0%),143 (7.9%),26 (1.4%),...,77 (4.2%),4 (0.2%),1 (0.1%),8 (0.4%),6 (0.3%),2 (0.1%),0 (0.0%),1 (0.1%),0 (0.0%),0 (0.0%)
Russia,141 (8.9%),5 (0.3%),146 (9.2%),138 (8.7%),177 (11.1%),55 (3.5%),137 (8.6%),11 (0.7%),106 (6.7%),7 (0.4%),...,4 (0.3%),11 (0.7%),1 (0.1%),2 (0.1%),5 (0.3%),5 (0.3%),7 (0.4%),0 (0.0%),0 (0.0%),0 (0.0%)
China,118 (11.2%),8 (0.8%),121 (11.5%),148 (14.1%),92 (8.8%),38 (3.6%),72 (6.9%),16 (1.5%),94 (9.0%),16 (1.5%),...,113 (10.8%),1 (0.1%),2 (0.2%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%)
Hong Kong,34 (8.2%),1 (0.2%),28 (6.8%),40 (9.7%),23 (5.6%),51 (12.3%),22 (5.3%),4 (1.0%),28 (6.8%),14 (3.4%),...,66 (15.9%),0 (0.0%),0 (0.0%),0 (0.0%),1 (0.2%),0 (0.0%),0 (0.0%),0 (0.0%),2 (0.5%),0 (0.0%)
United Kingdom,12 (3.3%),0 (0.0%),8 (2.2%),31 (8.4%),6 (1.6%),14 (3.8%),54 (14.7%),1 (0.3%),18 (4.9%),6 (1.6%),...,7 (1.9%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),1 (0.3%)
Germany,28 (8.2%),4 (1.2%),17 (5.0%),34 (9.9%),14 (4.1%),4 (1.2%),19 (5.6%),4 (1.2%),75 (21.9%),9 (2.6%),...,1 (0.3%),3 (0.9%),0 (0.0%),0 (0.0%),2 (0.6%),2 (0.6%),0 (0.0%),1 (0.3%),0 (0.0%),0 (0.0%)
France,27 (8.6%),2 (0.6%),13 (4.2%),35 (11.2%),19 (6.1%),7 (2.2%),46 (14.7%),6 (1.9%),32 (10.2%),0 (0.0%),...,2 (0.6%),0 (0.0%),0 (0.0%),0 (0.0%),1 (0.3%),0 (0.0%),0 (0.0%),0 (0.0%),1 (0.3%),0 (0.0%)
Canada,12 (3.8%),0 (0.0%),20 (6.4%),35 (11.2%),5 (1.6%),15 (4.8%),37 (11.8%),7 (2.2%),25 (8.0%),3 (1.0%),...,8 (2.6%),5 (1.6%),4 (1.3%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%)
Philippines,13 (4.4%),0 (0.0%),12 (4.1%),19 (6.4%),1 (0.3%),3 (1.0%),28 (9.5%),2 (0.7%),32 (10.8%),3 (1.0%),...,2 (0.7%),1 (0.3%),0 (0.0%),1 (0.3%),0 (0.0%),0 (0.0%),1 (0.3%),0 (0.0%),0 (0.0%),0 (0.0%)


Saved plot: data/annotation_analysis_v0/time_geography/geo_function_purpose_prompt.png

=== Geographic Distribution Table for interaction_features @ prompt ===


,None,Courtesy/Politeness,Role-assignment,Reinforcement/Praise/Scolding,Jailbreak attempt,Companionship,Reinforcement/Praise
Country,,,,,,,
United States,1539 (82.2%),131 (7.0%),146 (7.8%),23 (1.2%),23 (1.2%),11 (0.6%),0 (0.0%)
Russia,1398 (87.0%),68 (4.2%),104 (6.5%),17 (1.1%),13 (0.8%),5 (0.3%),1 (0.1%)
China,817 (75.7%),117 (10.8%),137 (12.7%),7 (0.6%),1 (0.1%),0 (0.0%),0 (0.0%)
Hong Kong,262 (60.0%),64 (14.6%),107 (24.5%),4 (0.9%),0 (0.0%),0 (0.0%),0 (0.0%)
United Kingdom,342 (93.2%),14 (3.8%),8 (2.2%),1 (0.3%),2 (0.5%),0 (0.0%),0 (0.0%)
Germany,281 (81.2%),25 (7.2%),12 (3.5%),22 (6.4%),4 (1.2%),2 (0.6%),0 (0.0%)
France,263 (83.8%),14 (4.5%),20 (6.4%),12 (3.8%),5 (1.6%),0 (0.0%),0 (0.0%)
Canada,263 (83.8%),20 (6.4%),21 (6.7%),5 (1.6%),5 (1.6%),0 (0.0%),0 (0.0%)
Philippines,276 (92.9%),12 (4.0%),6 (2.0%),0 (0.0%),3 (1.0%),0 (0.0%),0 (0.0%)


Saved plot: data/annotation_analysis_v0/time_geography/geo_interaction_features_prompt.png

=== Geographic Distribution Table for media_format @ prompt ===


...

Saved plot: data/annotation_analysis_v0/time_geography/geo_topic_turn.png

=== Geographic Distribution Table for sensitive_use_flags @ turn ===


,None,"Inciting violence, hateful or other harmful behavior (harassment & bullying)",Sexually explicit content (Other),"Inciting violence, hateful or other harmful behavior (physical harm)",Sexually explicit content (fictitious person),Sexually explicit content (real person),"Discriminatory practices (Misrepresentation, stereotyping, or inappropriate reference to sensitive attributes)","Possible presence of copyrighted, unreferenced material",Criminal planning or other suspected illegal activity not listed elsewhere,Privacy concerns (Possible sensitive information),...,Generating defamatory content,Privacy concerns (Possible identifiable information),Criminal planning or other suspected illegal activity not listed,"Cyberattacks (enabling/enacting malware, computer viruses, worms, malicious code, ...)","Inciting violence, hateful or other harmful behavior (self-harm)","Privacy concerns (Possible sensitive information) (e.g., API keys, passwords, other confidential information)",Cyberattacks,Potential violation of external policy / ethics,Impersonation attempts,Other
Country,,,,,,,,,,,,,,,,,,,,,
United States,1559 (85.9%),31 (1.7%),120 (6.6%),19 (1.0%),58 (3.2%),3 (0.2%),11 (0.6%),0 (0.0%),0 (0.0%),1 (0.1%),...,0 (0.0%),3 (0.2%),0 (0.0%),3 (0.2%),1 (0.1%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%)
Russia,1392 (87.3%),28 (1.8%),88 (5.5%),24 (1.5%),16 (1.0%),3 (0.2%),11 (0.7%),1 (0.1%),12 (0.8%),3 (0.2%),...,1 (0.1%),2 (0.1%),1 (0.1%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%)
China,1020 (98.0%),2 (0.2%),16 (1.5%),1 (0.1%),1 (0.1%),1 (0.1%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),...,0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%)
Hong Kong,404 (98.3%),0 (0.0%),5 (1.2%),1 (0.2%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),...,0 (0.0%),1 (0.2%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%)
United Kingdom,314 (85.8%),8 (2.2%),19 (5.2%),2 (0.5%),15 (4.1%),0 (0.0%),3 (0.8%),0 (0.0%),0 (0.0%),0 (0.0%),...,0 (0.0%),0 (0.0%),0 (0.0%),1 (0.3%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%)
Germany,278 (80.3%),24 (6.9%),29 (8.4%),4 (1.2%),7 (2.0%),1 (0.3%),1 (0.3%),0 (0.0%),0 (0.0%),0 (0.0%),...,0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),2 (0.6%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%)
France,239 (75.4%),3 (0.9%),42 (13.2%),0 (0.0%),5 (1.6%),7 (2.2%),9 (2.8%),1 (0.3%),1 (0.3%),1 (0.3%),...,3 (0.9%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%)
Canada,269 (87.6%),4 (1.3%),14 (4.6%),2 (0.7%),11 (3.6%),0 (0.0%),3 (1.0%),0 (0.0%),1 (0.3%),2 (0.7%),...,0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),1 (0.3%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%)
Philippines,288 (98.0%),1 (0.3%),2 (0.7%),0 (0.0%),1 (0.3%),0 (0.0%),1 (0.3%),1 (0.3%),0 (0.0%),0 (0.0%),...,0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%),0 (0.0%)


Saved plot: data/annotation_analysis_v0/time_geography/geo_sensitive_use_flags_turn.png

=== Geographic Distribution Table for model @ conversation ===


,gpt-3.5-turbo-0301,gpt-4-0314,gpt-3.5-turbo-0613,gpt-4-1106-preview,gpt-3.5-turbo-0125,gpt-4-0125-preview
Country,,,,,,
United States,105 (12.0%),26 (3.0%),449 (51.5%),138 (15.8%),83 (9.5%),71 (8.1%)
Russia,112 (18.4%),32 (5.2%),287 (47.0%),89 (14.6%),46 (7.5%),44 (7.2%)
China,148 (36.6%),16 (4.0%),186 (46.0%),8 (2.0%),40 (9.9%),6 (1.5%)
Hong Kong,24 (11.6%),4 (1.9%),135 (65.2%),12 (5.8%),29 (14.0%),3 (1.4%)
United Kingdom,18 (11.6%),2 (1.3%),69 (44.5%),24 (15.5%),13 (8.4%),29 (18.7%)
Germany,38 (27.5%),18 (13.0%),43 (31.2%),25 (18.1%),2 (1.4%),12 (8.7%)
France,17 (13.7%),10 (8.1%),51 (41.1%),33 (26.6%),1 (0.8%),12 (9.7%)
Canada,15 (16.5%),7 (7.7%),36 (39.6%),21 (23.1%),5 (5.5%),7 (7.7%)
Japan,10 (11.5%),2 (2.3%),45 (51.7%),8 (9.2%),16 (18.4%),6 (6.9%)


Saved plot: data/annotation_analysis_v0/time_geography/geo_model_conversation.png
